In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns

### Definindo caminhos para os arquivos

In [2]:

pd.set_option('display.max_columns', None)  # Mostrar todas as colunas
plt.style.use('ggplot')  # Estilo dos gráficos

# caminhos para os arquivos
base_path = "/home/iioulos/Documents/IC_Danilo-Cotozika/dados_Jaschke"
evaporacao = "/home/iioulos/Documents/IC_Danilo-Cotozika/dados_Jaschke/EVAPORAÇÃO_CLASSE_A"
precipitacao = "/home/iioulos/Documents/IC_Danilo-Cotozika/dados_Jaschke/precipitacao_1968_2002"
tabelas2006 = "/home/iioulos/Documents/IC_Danilo-Cotozika/dados_Jaschke/TABELAS 2006/TABELAS"
tabelas_2 = "/home/iioulos/Documents/IC_Danilo-Cotozika/dados_Jaschke/TABELAS 2006/TABELAS/TABELAS_2"
tabelas_gerais = "/home/iioulos/Documents/IC_Danilo-Cotozika/dados_Jaschke/TABELAS GERAIS/TABELAS"
TAB_PRECIPITACAO_1968_2002 = "/home/iioulos/Documents/IC_Danilo-Cotozika/dados_Jaschke/TAB_PRECIPITAÇÃO_1968_2002"
TAB_TEMPERATURA_1968_2002 = "/home/iioulos/Documents/IC_Danilo-Cotozika/dados_Jaschke/TAB_TEMPERATURA_1968_2002"
ventos = "/home/iioulos/Documents/IC_Danilo-Cotozika/dados_Jaschke/VENTOS"


def listar_arquivos_xls(caminho):
    """Lista todos os arquivos .xls no diretório especificado"""
    arquivos = [f for f in os.listdir(caminho) if f.endswith('.xls') or f.endswith('.xlsx')]
    print(f"Encontrados {len(arquivos)} arquivos .xls/.xlsx no diretório:")
    for i, arquivo in enumerate(arquivos, 1):
        print(f"{i}. {arquivo}")
    return arquivos


### Passando o caminho e listando os arquivos

In [3]:
caminho = evaporacao
arquivos = listar_arquivos_xls(caminho)

Encontrados 3 arquivos .xls/.xlsx no diretório:
1. TEMPERATURA_AGUA_CLASSE_A.xls
2. EVAPORACAO_CLASSE_A.xls
3. CLASSE_A_80-85.xls


### Função para carregar e analisar um xml

In [4]:
def analisar_arquivo(nome_arquivo, caminho):
    """Carrega e analisa um arquivo XLS, retornando informações básicas"""
    caminho_completo = os.path.join(caminho, nome_arquivo)
    print(f"\n{'='*50}")
    print(f"Analisando arquivo: {nome_arquivo}")
    
    try:
        # Tentar ler o arquivo (pode ser .xls ou .xlsx)
        df = pd.read_excel(caminho_completo, header=1)
        
        # Informações básicas
        print(f"\nShape: {df.shape} (linhas, colunas)")
        print("\nPrimeiras linhas:")
        print(df.head())
        
        print("\nInformações do DataFrame:")
        print(df.info())
        
        print("\nEstatísticas descritivas:")
        print(df.describe(include='all'))
        
        # Verificar valores nulos
        print("\nValores nulos por coluna:")
        print(df.isnull().sum())
        
        return df
        
    except Exception as e:
        print(f"Erro ao ler o arquivo {nome_arquivo}: {str(e)}")
        return None

### Transformando o arquivo.xls num dataframe

In [5]:
nome_arquivo = 'EVAPORACAO_CLASSE_A.xls'
#df = analisar_arquivo(nome_arquivo, caminho)
df = pd.read_excel(os.path.join(caminho, nome_arquivo), header=1)
# Drop the column 'Unnamed: 13' if it exists
if 'Unnamed: 13' in df.columns:
	df = df.drop(columns=['Unnamed: 13'])

# Rename the column 'RESUMO MENSAL' to an empty string if it exists
if 'RESUMO MENSAL' in df.columns:
	df = df.rename(columns={'RESUMO MENSAL': ''})

# Drop the column 'Unnamed: 15' if it exists
if 'Unnamed: 15' in df.columns:
	df = df.rename(columns={'Unnamed: 15':'Resumo Mensal'})
 
df.head(10)

,dia,jan,fev,mar,abr,mai,jun,jul,ago,set,out,nov,dez,,Resumo Mensal
0,1,nd,nd,3.15,4.54,nd,2.6,nd,3.36,5.8,8.57,8.56,5.54,ANO:,1980
1,2,nd,7.78,9.42,nd,1.5,0.37,0.77,5.53,6.71,9.61,6.44,5.57,MÊS,TOTAL
2,3,nd,7.97,7.38,0,0.92,2.21,1.07,6.81,5.66,8.77,6.87,4.84,janeiro,94.94
3,4,nd,13.16,7.03,0,6.39,6.86,2.24,5.1,2.91,6.45,9.2,5.23,fevereiro,131.16
4,5,nd,2.04,4.39,0.3,4.05,7.79,3.39,3.41,8.63,7.61,7,8.35,março,153.91
5,6,nd,6,0.9,9.35,3.9,3.66,3.14,2.07,6.22,8.23,4.44,7.89,abril,106.61
6,7,nd,3.22,11.11,4.1,4.83,3.87,4.6,3.71,6.61,2.24,4.76,9.77,maio,93.63
7,8,4.02,8.96,13.78,1.94,4.28,4.13,4.3,2.29,nd,2.64,2.4,9.53,junho,95.21
8,9,0.66,nd,nd,3.66,5.54,3.45,2.91,5.07,2.81,6.09,6.01,8.1,julho,121.88
9,10,nd,nd,5.56,4.04,nd,4.17,2.31,1.41,2.69,nd,1.49,7.1,agosto,148.35


### Funções para visualisar os dados

In [6]:
def visualizar_dados(df, nome_arquivo, max_cols):
    """Gera visualizações básicas dos dados"""
    if df is None or df.empty:
        return
    
    print(f"\nVisualizando dados de: {nome_arquivo}")
    
    # Selecionar apenas colunas numéricas para visualização
    numeric_cols = df.select_dtypes(include=['number']).columns
    
    if len(numeric_cols) == 0:
        print("Nenhuma coluna numérica encontrada para visualização.")
        return
    
    # Limitar o número de colunas para evitar muitos gráficos
    if len(numeric_cols) > max_cols:
        numeric_cols = numeric_cols[:max_cols]
        print(f"Mostrando apenas as primeiras {max_cols} colunas numéricas...")
    
    # Histogramas
    df[numeric_cols].hist(bins=20, figsize=(15, 10))
    plt.suptitle(f"Histogramas - {nome_arquivo}", y=1.02)
    plt.tight_layout()
    plt.show()
    
    # Boxplots
    plt.figure(figsize=(15, 5))
    df[numeric_cols].boxplot()
    plt.title(f"Boxplots - {nome_arquivo}")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Matriz de correlação (se houver colunas suficientes)
    if len(numeric_cols) > 1:
        plt.figure(figsize=(10, 8))
        sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm', center=0)
        plt.title(f"Matriz de Correlação - {nome_arquivo}")
        plt.tight_layout()
        plt.show()


## Visualisando os dados

In [7]:
max_cols = 10
visualizar_dados(df, nome_arquivo, max_cols)


Visualizando dados de: EVAPORACAO_CLASSE_A.xls
Nenhuma coluna numérica encontrada para visualização.


### Padronizando os dados:

In [8]:
def padronizar_dados(df):
    
    # Converte todos os nomes das colunas para strings
    df.columns = df.columns.astype(str)
    """Padroniza os dados numéricos usando StandardScaler"""
    numeric_cols = df.select_dtypes(include=['number']).columns
    
    if len(numeric_cols) == 0:
        print("Nenhuma coluna numérica encontrada para padronização.")
        return df
    
    scaler = StandardScaler()
    df_padronizado = df.copy()
    df_padronizado[numeric_cols] = scaler.fit_transform(df_padronizado[numeric_cols])
    
    print("\nDados padronizados (primeiras linhas):")
    print(df_padronizado.head())
    
    return df_padronizado